In [ ]:
import cv2
import numpy as np
from keras.models import model_from_json

In [ ]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# load json and create model
json_file = open('/content/drive/MyDrive/Human Activity Recognition/Emotion_detection_with_CNN-main/model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

In [ ]:
# load weights into new model
emotion_model.load_weights("/content/drive/MyDrive/Human Activity Recognition/Emotion_detection_with_CNN-main/model/emotion_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
from google.colab import output
from PIL import Image
from google.colab.patches import cv2_imshow


# # Install required packages (if not already installed)
# !pip install opencv-python-headless
# !pip install numpy

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Pass the video path
cap = cv2.VideoCapture("/content/drive/MyDrive/Human Activity Recognition/Emotion_detection_with_CNN-main/Emotion.mov")

# Get the input video dimensions
width = int(cap.get(3))
height = int(cap.get(4))

# Set the output video file path
output_path = "/content/drive/MyDrive/Human Activity Recognition/Emotion_detection_with_CNN-main/output_video.mp4"

# Define the codec and create a VideoWriter object with the same dimensions as the input video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 20.0, (width, height))  # Use the dimensions of the input video

while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    if not ret:
        break  # Break out of the loop when there are no more frames

    frame = cv2.resize(frame, (width, height))
    face_detector = cv2.CascadeClassifier('/content/drive/MyDrive/Human Activity Recognition/Emotion_detection_with_CNN-main/haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    # Display the frame
    cv2_imshow(frame)

    # Write the frame to the output video
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and VideoWriter objects
cap.release()
out.release()
cv2.destroyAllWindows()
